<a href="https://colab.research.google.com/github/paulhutchings/midi-nn/blob/note_sequences/Midi_NN_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook contains an Exploratory Data Analysis (EDA) of the International Piano e-Competition dataset(IPeC). The notebook guides the user through downloading the dataset from Github, converting the MIDI files to NoteSequences for easier use, and performing a number of data operations to get an insight into the MIDI data contained in the dataset.

In [ ]:
# install dependencies
!pip3 install note-seq

In [9]:
# download the dataset hosted on Github and extract the files
!mkdir dataset && git clone https://github.com/paulhutchings/international-e-piano-dataset dataset
!cat dataset/international-e-piano-midi_2002-2018.tar.gz.parta* > dataset/midi.tar.gz
!mkdir data data/midi data/notesequences && tar -xf dataset/midi.tar.gz -C data/midi
!ls data

Now, we will convert all of the MIDI files into NoteSequences, a serialized data structure used by Google's Magenta project which is much easier to work with than raw MIDI files.